In [ ]:
# | default_exp lobby

In [ ]:
# | export
import datetime as dt
import logging
from dataclasses import field
import random, string
from fasthtml.common import *
from meme_games.utils import *
from meme_games.user import *

In [ ]:
#| export
logger = logging.getLogger(__name__)

In [ ]:
# | export
def random_id(size=5): return ''.join(random.choices(string.ascii_lowercase+string.digits, k=size))

In [ ]:
# | export

@dataclass
class LobbyMember:
    user: User
    ws: Optional[FunctionType] = None
    joined_at: dt.datetime = field(default_factory=dt.datetime.now)

    def __post_init__(self): self.uid = self.user.uid
    def connect(self, ws): self.ws = ws

    @property
    def is_connected(self): return self.ws is not None


@dataclass
class Lobby:
    id: str
    host: LobbyMember
    members: dict[str, LobbyMember] = field(default_factory=dict)
    last_active: dt.datetime = field(default_factory=dt.datetime.now)

    def __post_init__(self):
        if self.host.uid not in self.members:
            self.members[self.host.uid] = self.host

    def get_or_create_member(self, user: User, ws: FunctionType = None):
        u = self.members.setdefault(user.uid, LobbyMember(user))
        u.ws = ws
        return u

In [ ]:
# | export

class LobbyManager:
    def __init__(self):
        self.lobbies = {}

    def __repr__(self): return f'{self.__class__.__name__}(active_lobbies={len(self.lobbies)})'

    def create_lobby(self, u: User):
        lobby_id = random_id()
        while lobby_id in self.lobbies:
            lobby_id = random_id()
        lobby = Lobby(lobby_id, LobbyMember(u))
        self.lobbies[lobby_id] = lobby
        return lobby

    def get_lobby(self, id: Optional[str] = None) -> Optional[Lobby]: return self.lobbies.get(id)
    def delete_lobby(self, id: str): self.lobbies.pop(id)

In [ ]:
db = init_db()
user_manager = UserManager(db)
user = user_manager.create()
manager = LobbyManager()

In [ ]:
lobby = manager.create_lobby(user)

## Views

In [ ]:
# | export
def Members(lobby: Lobby):
    return Table(
        Tr(Th('Name'), Th('id'), Th('Is connected')),
        *[Tr(Td(UserName(member.user)), Td(member.uid), Td(member.is_connected)) for member in lobby.members.values()],
        id='members'
    )